In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import os
import math
import datetime
import matplotlib.pyplot as plt 
import scipy.stats as sstats
from scipy.stats.sampling import DiscreteAliasUrn
from matplotlib import cm
import random
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import numpy
%matplotlib inline 
from matplotlib import pyplot as plt
from PIL import Image
import requests
from io import BytesIO
import IPython.display
import json
import sys
import yaml
from random import randrange
from functions_gapfill import *


In [ ]:
# we want to introdce 10% missing data at locations X and 
# 24, 48,72,96

# Gapfilling the AWS of the LéXPLORE platform

In this notebook we use the G2S server with Direct Sampling approach (https://gaia-unil.github.io/G2S/briefOverview.html) to fill the data gaps of the meteo station or AWS of the LéXPLORE platform (https://gitlab.renkulab.io/lexplore).

See other notebooks in this repository on how to arrive at this point. 

We use already 1 hourly aggregated values of the meteo station and we fill gaps for the following variables on 1 hour resolution:

* Air temperature

* Baromatric pressure 

* Relative Humidity

* Wind Speed

* Wind Direction

* Wind Gusts

* Precipitation

* Solar Incoming Radiation 

* Solar Total Incoming Radation (PROBABLY NOT)

To do so, we use independent data as co-variates, namely variables from the closest gridpoint in ERA5 and ERA5-land. Other potential co-variates could be AWS data from other locations around Lake Geneva or data from other weather models or reconstructions. 

## Activate G2S server

In [ ]:
#!pip install G2S libtiff --quiet
from g2s import g2s
g2s('--version')


In [ ]:
#! git clone https://github.com/GAIA-UNIL/G2S.git --quiet

In [ ]:
#%%capture
#!export NVFLAGS='-gencode=arch=compute_35,code=sm_35 -gencode=arch=compute_37,code=sm_37 -gencode=arch=compute_50,code=sm_50 -gencode=arch=compute_52,code=sm_52 -gencode=arch=compute_60,code=sm_60 -gencode=arch=compute_61,code=sm_61 -gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_70,code=compute_70'
#!sudo apt -qq install build-essential libzmq3-dev libjsoncpp-dev zlib1g-dev libfftw3-dev libcurl4-openssl-dev -y
#!sudo wget -q "https://raw.githubusercontent.com/zeromq/cppzmq/master/zmq.hpp" -O /usr/include/zmq.hpp
#!( cd G2S/build && make c++ -j --silent)
#!bash G2S/build/c++-build/install_needs_W_VM.sh

In [ ]:
!pwd

In [ ]:
os.chdir("/home/mwegmann/g2s")

In [ ]:
!bash -c "cd G2S/build/c++-build/ && ./server -d"


In [ ]:
ti = numpy.array(Image.open(BytesIO(requests.get('https://raw.githubusercontent.com/GAIA-UNIL/TrainingImagesTIFF/master/stone.tiff').content)));

In [ ]:
a=g2s('-a','echo','-ti',ti,'-dt',[0])

In [ ]:
plt.imshow(a[0], interpolation='nearest')

## folder setup

In [ ]:
# change yaml location here
with open(r"/home/mwegmann/g2s/notebooks/folder_gap_filling.yaml", "r") as f:
    directories = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
#for d in directories.values():
#    if not os.path.exists(d):
#        os.makedirs(d)

In [ ]:
# defining folders
input_folder=directories["g2s_input_folder"]

output_folder=directories["g2s_output_folder"]

scripts_folder=directories["scripts_folder"]

## read in data

In [ ]:
meteo_orig=xr.open_dataset(input_folder+"meteo_1hr_g2s.nc")

In [ ]:
meteo_orig

In [ ]:
era5=xr.open_dataset(input_folder+"era5_lexplore_g2s.nc")

In [ ]:
era5

In [ ]:
era5_land=xr.open_dataset(input_folder+"era5_land_lexplore_g2s.nc")

In [ ]:
era5_land

## Fill Air Temperature Data

### understand the average gap size in data

In [ ]:
varname="AirTC"


In [ ]:
gap_info(var=meteo_orig[varname],varname="meteo_orig_airtc",plot_folder=output_folder)

### Run gapfilling

The idea is to create gaps that are always at the same position, but different in size.


It is tricky to think about a "perfect gap size" that we should try out. I would suggest that we try out gap sizes from 1 day (24 missing values in the case of the meteo station) towards 4 days (96 missing values in case of the meteo station).

The other question is how many (in %) new missing values do we introduce to evaluate our routine. The meteo station has 3% missing data as it is. Is 10% a good metric to evaluate? 20%?

We have three steps for the reconstruction of each variable:

* Covariate evaluation with N=25, 5% missing data, and [24,48,72] gaps and 10 test runs

* Creating error matrix for error propagation analysis with preferred covariate with N=25, [5,10,15,20] missing data, and [24,48,72,96] gaps and 5 test runs

* Creating the final product/reconstruction with N=50

#### eval phase

In [ ]:
percent_list=[5]
gap_amount_list=[24,48,72]
selector_list=[1,2,3]
#gap_amount_list=[150]
N = 25
test_runs=10


columns_df=["NAME",'RUN',"MEMBER","PERC","GAP_SIZE","CORR","RMSE","STDR"]

df = pd.DataFrame(columns=columns_df)

In [ ]:
print(datetime.datetime.now())

In [ ]:
filled_data,error_df=univ_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=day_of_year_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=time_of_day_of_year_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5,var2="t2m",cov_name="era5",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5_land,var2="t2m",cov_name="era5l",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
error_df

In [ ]:
error_df=pd.read_csv(output_folder+"meteo_eval_"+varname+".csv")


In [ ]:
error_df[error_df.NAME =="era5l"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="era5"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="UV"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="calday"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="caldaytimeday"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

#### error matrix phase

In [ ]:
percent_list=[5,10,15,20]
gap_amount_list=[24,48,72,96]
selector_list=[1,2,3,4]
#gap_amount_list=[150]
N = 25
test_runs=5


columns_df=["NAME",'RUN',"MEMBER","PERC","GAP_SIZE","CORR","RMSE","STDR"]

df = pd.DataFrame(columns=columns_df)

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5,var2="t2m",cov_name="era5",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_era5_")

In [ ]:
error_df=pd.read_csv(output_folder+"meteo_era5_"+varname+".csv")


In [ ]:
error_df[error_df.columns[1:]].groupby('PERC').mean()

In [ ]:
error_df[error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
perc=error_df["PERC"].values
gap_size=error_df["GAP_SIZE"].values
rmse=error_df["RMSE"].values

In [ ]:
import seaborn as sns
sns.relplot(data=error_df, x='PERC', y='GAP_SIZE', hue='RMSE')
plt.show()

In [ ]:
import seaborn as sns
sns.relplot(data=error_df, x='PERC', y='GAP_SIZE', hue='CORR')
plt.show()

In [ ]:
# importing mplot3d toolkits, numpy and matplotlib
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
 
fig = plt.figure()
 
# syntax for 3-D projection
ax = plt.axes(projection ='3d')
 
# defining all 3 axis
z = rmse
x = gap_size
y = perc
 
# plotting
ax.plot3D(x, y, z, '.b')
ax.set_title('3D line plot geeks for geeks')
plt.show()

#### reconstruction phase

In [ ]:
N = 50

In [ ]:
#gap-filling with one covariate
covar2 = era5["t2m"].copy()
#name_addedinfo=cov_name
#covar2.loc[dict(time = covar2.time[gap_indices])] = np.nan
gapped_data=meteo_orig[varname].copy()

ti = np.stack([gapped_data.data,
            covar2],axis = 1)
di = np.stack([gapped_data.data,
            covar2],axis = 1)
dt = [0,0,] 
#ki = np.ones([L,5])
#ki[:,:4] = 0.3 #Assign half weight to categorical variable 


stacked = ensemble_QS(N = N,
                      ti=ti, 
                      di=di,
                      dt=dt, #Zero for continuous variables
                      k=1.2,
                      n=50,
                      j=0.5,
                      ki=None)
simulation = xr.DataArray(data =stacked[:,:,0],
                            coords = {'realizations':np.arange(1,stacked.shape[0]+1),'time':gapped_data.time})

In [ ]:
year = 2021
start_month =3
end_month = 5

plot_MPS_ensembles(original = gapped_data,
                   simulation = simulation, 
                   year = year,
                   start_month = start_month,
                   end_month = end_month,plot_folder=output_folder,
                  alpha = 0.1,
                  title = "meteo_"+varname+"AirTC_era5_simulation")

In [ ]:
save_name="meteo_"+varname+"_era5_50member_reconstruction.nc"

simulation.to_netcdf(output_folder+save_name)

xr.open_dataset(output_folder+save_name)

## Fill Air Pressure Data

### understand the average gap size in data

In [ ]:
varname="BP"


In [ ]:
gap_info(var=meteo_orig[varname],varname="meteo_orig_bp",plot_folder=output_folder)

### Run gapfilling

In [ ]:
percent_list=[5]
gap_amount_list=[24,48,72]
selector_list=[1,2,3]
#gap_amount_list=[150]
N = 25
test_runs=10


columns_df=["NAME",'RUN',"MEMBER","PERC","GAP_SIZE","CORR","RMSE","STDR"]

df = pd.DataFrame(columns=columns_df)

In [ ]:
print(datetime.datetime.now())

In [ ]:
filled_data,error_df=univ_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=day_of_year_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=time_of_day_of_year_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5,var2="sp",cov_name="era5",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5_land,var2="sp",cov_name="era5l",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
error_df

In [ ]:
error_df=pd.read_csv(output_folder+"meteo_eval_"+varname+".csv")


In [ ]:
error_df[error_df.NAME =="era5l"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="era5"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="UV"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="calday"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="caldaytimeday"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

#### error matrix phase

In [ ]:
percent_list=[5,10,15,20]
gap_amount_list=[24,48,72,96]
selector_list=[1,2,3,4]
#gap_amount_list=[150]
N = 25
test_runs=5


columns_df=["NAME",'RUN',"MEMBER","PERC","GAP_SIZE","CORR","RMSE","STDR"]

df = pd.DataFrame(columns=columns_df)

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5,var2="sp",cov_name="era5",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_era5_")

In [ ]:
error_df=pd.read_csv(output_folder+"meteo_era5_"+varname+".csv")


In [ ]:
error_df[error_df.columns[1:]].groupby('PERC').mean()

In [ ]:
error_df[error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
perc=error_df["PERC"].values
gap_size=error_df["GAP_SIZE"].values
rmse=error_df["RMSE"].values

In [ ]:
import seaborn as sns
sns.relplot(data=error_df, x='PERC', y='GAP_SIZE', hue='RMSE')
plt.show()

In [ ]:
import seaborn as sns
sns.relplot(data=error_df, x='PERC', y='GAP_SIZE', hue='CORR')
plt.show()

In [ ]:
# importing mplot3d toolkits, numpy and matplotlib
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
 
fig = plt.figure()
 
# syntax for 3-D projection
ax = plt.axes(projection ='3d')
 
# defining all 3 axis
z = rmse
x = gap_size
y = perc
 
# plotting
ax.plot3D(x, y, z, '.b')
ax.set_title('3D line plot geeks for geeks')
plt.show()

#### reconstruction phase

In [ ]:
N = 50

In [ ]:
#gap-filling with one covariate
covar2 = era5["sp"].copy()
#name_addedinfo=cov_name
#covar2.loc[dict(time = covar2.time[gap_indices])] = np.nan
gapped_data=meteo_orig[varname].copy()

ti = np.stack([gapped_data.data,
            covar2],axis = 1)
di = np.stack([gapped_data.data,
            covar2],axis = 1)
dt = [0,0,] 
#ki = np.ones([L,5])
#ki[:,:4] = 0.3 #Assign half weight to categorical variable 


stacked = ensemble_QS(N = N,
                      ti=ti, 
                      di=di,
                      dt=dt, #Zero for continuous variables
                      k=1.2,
                      n=50,
                      j=0.5,
                      ki=None)
simulation = xr.DataArray(data =stacked[:,:,0],
                            coords = {'realizations':np.arange(1,stacked.shape[0]+1),'time':gapped_data.time})

In [ ]:
year = 2021
start_month =3
end_month = 5

plot_MPS_ensembles(original = gapped_data,
                   simulation = simulation, 
                   year = year,
                   start_month = start_month,
                   end_month = end_month,plot_folder=output_folder,
                  alpha = 0.1,
                  title = "meteo_"+varname+"_era5_simulation")

In [ ]:
save_name="meteo_"+varname+"_era5_50member_reconstruction.nc"

simulation.to_netcdf(output_folder+save_name)

xr.open_dataset(output_folder+save_name)

## Fill Incoming Solar Radiation Data

### understand the average gap size in data

In [ ]:
varname="Slrw"


In [ ]:
gap_info(var=meteo_orig[varname],varname="meteo_orig_slrw",plot_folder=output_folder)

### Run gapfilling

In [ ]:
percent_list=[5]
gap_amount_list=[24,48,72]
selector_list=[1,2,3]
#gap_amount_list=[150]
N = 25
test_runs=10


columns_df=["NAME",'RUN',"MEMBER","PERC","GAP_SIZE","CORR","RMSE","STDR"]

df = pd.DataFrame(columns=columns_df)

In [ ]:
print(datetime.datetime.now())

In [ ]:
filled_data,error_df=univ_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=day_of_year_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=time_of_day_of_year_g2s(original=meteo_orig,var=varname,obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5,var2="ssrd",cov_name="era5",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5_land,var2="ssrd",cov_name="era5l",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_eval_")

In [ ]:
error_df

In [ ]:
error_df=pd.read_csv(output_folder+"meteo_eval_"+varname+".csv")


In [ ]:
error_df[error_df.NAME =="era5l"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="era5"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="UV"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="calday"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
error_df[error_df.NAME =="caldaytimeday"][error_df.columns[1:]].groupby('GAP_SIZE').mean()

#### error matrix phase

In [ ]:
percent_list=[5,10,15,20]
gap_amount_list=[24,48,72,96]
selector_list=[1,2,3,4]
#gap_amount_list=[150]
N = 25
test_runs=5


columns_df=["NAME",'RUN',"MEMBER","PERC","GAP_SIZE","CORR","RMSE","STDR"]

df = pd.DataFrame(columns=columns_df)

In [ ]:
filled_data,error_df=one_cov_g2s(original=meteo_orig,var1=varname,cov=era5,var2="ssrd",cov_name="era5",obs_in_day=24,N=N,percent_list=percent_list,gap_amount_list=gap_amount_list,selector_list=selector_list,test_runs=test_runs,df=df,csv_folder=output_folder,name="meteo_era5_")

In [ ]:
error_df=pd.read_csv(output_folder+"meteo_era5_"+varname+".csv")


In [ ]:
error_df[error_df.columns[1:]].groupby('PERC').mean()

In [ ]:
error_df[error_df.columns[1:]].groupby('GAP_SIZE').mean()

In [ ]:
perc=error_df["PERC"].values
gap_size=error_df["GAP_SIZE"].values
rmse=error_df["RMSE"].values

In [ ]:
import seaborn as sns
sns.relplot(data=error_df, x='PERC', y='GAP_SIZE', hue='RMSE')
plt.show()

In [ ]:
import seaborn as sns
sns.relplot(data=error_df, x='PERC', y='GAP_SIZE', hue='CORR')
plt.show()

In [ ]:
# importing mplot3d toolkits, numpy and matplotlib
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
 
fig = plt.figure()
 
# syntax for 3-D projection
ax = plt.axes(projection ='3d')
 
# defining all 3 axis
z = rmse
x = gap_size
y = perc
 
# plotting
ax.plot3D(x, y, z, '.b')
ax.set_title('3D line plot geeks for geeks')
plt.show()

#### reconstruction phase

In [ ]:
N = 50

In [ ]:
#gap-filling with one covariate
covar2 = era5["sp"].copy()
#name_addedinfo=cov_name
#covar2.loc[dict(time = covar2.time[gap_indices])] = np.nan
gapped_data=meteo_orig[varname].copy()

ti = np.stack([gapped_data.data,
            covar2],axis = 1)
di = np.stack([gapped_data.data,
            covar2],axis = 1)
dt = [0,0,] 
#ki = np.ones([L,5])
#ki[:,:4] = 0.3 #Assign half weight to categorical variable 


stacked = ensemble_QS(N = N,
                      ti=ti, 
                      di=di,
                      dt=dt, #Zero for continuous variables
                      k=1.2,
                      n=50,
                      j=0.5,
                      ki=None)
simulation = xr.DataArray(data =stacked[:,:,0],
                            coords = {'realizations':np.arange(1,stacked.shape[0]+1),'time':gapped_data.time})

In [ ]:
year = 2021
start_month =3
end_month = 5

plot_MPS_ensembles(original = gapped_data,
                   simulation = simulation, 
                   year = year,
                   start_month = start_month,
                   end_month = end_month,plot_folder=output_folder,
                  alpha = 0.1,
                  title = "meteo_"+varname+"_era5_simulation")

In [ ]:
save_name="meteo_"+varname+"_era5_50member_reconstruction.nc"

simulation.to_netcdf(output_folder+save_name)

xr.open_dataset(output_folder+save_name)